## 1. Install necessary packages

In [2]:
! pip install azureml-sdk[automl,explain]==1.0.43 matplotlib onnx opencv-python

## 2. Load workspace from conf.json

In [2]:
import azureml.core
import pandas as pd
import numpy as np
import logging
import warnings
# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None


from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig


In [3]:
ws = Workspace.from_config()

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
SDK version,1.0.43
Subscription ID,ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9
Workspace,zhanxiaAML
Resource Group,MT
Location,eastus


## 3. Print the exiting compute in the workspace

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

compute = ws.compute_targets
compute

{'TryNotenook': <azureml.core.compute.dsvm.DsvmCompute at 0x7f0a6db53518>,
 'DefaultAKS': <azureml.core.compute.aks.AksCompute at 0x7f0a6db53630>,
 'aml-compute': <azureml.core.compute.amlcompute.AmlCompute at 0x7f0a6db537b8>,
 'cpucluster': <azureml.core.compute.amlcompute.AmlCompute at 0x7f0a6db538d0>,
 'gpucluster': <azureml.core.compute.amlcompute.AmlCompute at 0x7f0a6db53ac8>}

## 4. Upload data for remote compute to access

In [5]:
ds = ws.get_default_datastore()
ds.upload_files(['./data/nyc_energy.csv'], target_path='exp_data', overwrite=True, show_progress=True)

from azureml.core.runconfig import DataReferenceConfiguration
dr = DataReferenceConfiguration(datastore_name=ds.name, 
                   path_on_datastore='exp_data', 
                   path_on_compute='/tmp/azureml_runs',
                   mode='download', # download files from datastore to compute target
                   overwrite=False)

Uploading ./data/nyc_energy.csv
Uploaded ./data/nyc_energy.csv, 1 files out of an estimated total of 1
